In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
groq_api_key = os.getenv("GROQ_API_KEY")
gemini_api_key = os.getenv("GEMINI_API_KEY")

#### *document preprocessing*

In [4]:
from langchain.document_loaders import TextLoader

loader = TextLoader('extracted_content.txt',encoding='utf-8')

In [5]:
docs = loader.load()

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)

docs = splitter.split_documents(docs)

In [7]:
docs

[Document(metadata={'source': 'extracted_content.txt'}, page_content='At Capillary, we offer AI-based cloud-native SaaS products and solutions such as automated loyalty management and consumer data platform. Our diverse product range enables you to manage end-to-end loyalty programs, gain a comprehensive view of your consumers, and offer unified, cross-channel strategies that provide consumers with real-time, omnichannel, personalized, and consistent experience.\n\nLoyalty+ - Lets you increase repeat sales with personalized, omnichannel loyalty programs by intelligently rewarding your customers for desired behavior. Know more details.\nEngage+ - This allows you to personalize customer engagement with omnichannel retail marketing solutions.\nCustomer Data Platform (CDP): An effective platform to ingest data from different sources and platforms. There are several products to help capture brand data from different sources. Know more details.\nInsights+ - Provides you with profitable insig

#### *vectorDB and retriever*

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\SHIVAM GHUGE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


c:\Users\SHIVAM GHUGE\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
from langchain_chroma import Chroma

vectordb = Chroma.from_documents(documents=docs,embedding=embeddings)

In [10]:
retriever = vectordb.as_retriever(
    search_kwargs = {'k':4}
)

#### *Gemini model and Response*

In [11]:
import google.generativeai as genai

genai.configure(api_key=gemini_api_key)

In [12]:
for model in genai.list_models():
    if 'generateContent' in model.supported_generation_methods:
        print(model.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinki

In [13]:
config = genai.GenerationConfig(temperature=0.6)

model = genai.GenerativeModel(model_name="gemini-1.5-flash",generation_config=config)

In [14]:
def get_prompt(context,question):
    prompt = f'''
        You are an intelligent virtual assistant with expertise in understanding and processing documentation across various domains. 
        Your speciality lies in extracting relevant information from provided {context} and delivering clear, concise, and informative responses based on user queries.
        
        Answer the {question} posed by users regarding the context given to you.
        If the user wanted any piece of information in any structured format make sure to provide it.
    '''
    return prompt

In [15]:
user_query = input()

In [16]:
context = retriever.invoke(user_query)

In [17]:
context

[Document(metadata={'source': 'extracted_content.txt'}, page_content='Login with Google Credentials\nIn a browser, open the URL of your cluster.\nEU Cluster - https://eu.intouch.capillarytech.com/auth/login\nSG Cluster - https://apac2.intouch.capillarytech.com/auth/login\nIndia Cluster - https://intouch.capillary.co.in/auth/login\nClick on Sign in with Google as shown below.\nEnter the email ID your Capillary Google Workspace account and click Next.\nEnter your password and click Next.\n\nYou will see Intouch Workbench. If you have already signed in to your Capillary Google Workspace account, you will directly see the Workbech page.\n\nLogin with SSO\n\nSign in with a single set of credentials used for multiple applications provided by third party like Microsoft ADFS, G Suite SAML. It allows brand users to login to Capillary Intouch using their Identity Provider (IdP).\n\nSetup SSO with Okta\nWe have partnered with Okta for the integration. SSO setup requires configuring a connection f

In [18]:
response = model.generate_content(
    contents=get_prompt(context,user_query)
)

In [19]:
print(response.text)

To log in to Capillary Intouch using Google credentials:

1. **Open the correct URL:** Go to the URL for your cluster in a web browser:
    * **EU Cluster:** `https://eu.intouch.capillarytech.com/auth/login`
    * **SG Cluster:** `https://apac2.intouch.capillarytech.com/auth/login`
    * **India Cluster:** `https://intouch.capillary.co.in/auth/login`

2. **Click "Sign in with Google":**  Look for the "Sign in with Google" button on the login page.

3. **Enter your Capillary Google Workspace email:** Enter the email address associated with your Capillary Google Workspace account.

4. **Enter your password:** Enter the password for your Capillary Google Workspace account.

5. **Access Intouch Workbench:** After successful authentication, you will be directed to the Intouch Workbench.  If you're already signed in to your Capillary Google Workspace account, you'll go directly to the Workbench page.


**Important Note:** You must have a Capillary Google Workspace account (an email address w

#### *Evaluation*

In [40]:
from datasets import Dataset

data = [
    {
        "question" : user_query,
        "answer" : response.text,
        "retrieved_contexts" : [doc.page_content for doc in context]
    }
]

In [49]:
[doc.page_content for doc in context]

['Login with Google Credentials\nIn a browser, open the URL of your cluster.\nEU Cluster - https://eu.intouch.capillarytech.com/auth/login\nSG Cluster - https://apac2.intouch.capillarytech.com/auth/login\nIndia Cluster - https://intouch.capillary.co.in/auth/login\nClick on Sign in with Google as shown below.\nEnter the email ID your Capillary Google Workspace account and click Next.\nEnter your password and click Next.\n\nYou will see Intouch Workbench. If you have already signed in to your Capillary Google Workspace account, you will directly see the Workbech page.\n\nLogin with SSO\n\nSign in with a single set of credentials used for multiple applications provided by third party like Microsoft ADFS, G Suite SAML. It allows brand users to login to Capillary Intouch using their Identity Provider (IdP).\n\nSetup SSO with Okta\nWe have partnered with Okta for the integration. SSO setup requires configuring a connection for Capillary in Org’s IdP and setting up Org’s Identity in Okta Admi

In [41]:
data = Dataset.from_list(data)

In [48]:
data['retrieved_contexts']

[['Login with Google Credentials\nIn a browser, open the URL of your cluster.\nEU Cluster - https://eu.intouch.capillarytech.com/auth/login\nSG Cluster - https://apac2.intouch.capillarytech.com/auth/login\nIndia Cluster - https://intouch.capillary.co.in/auth/login\nClick on Sign in with Google as shown below.\nEnter the email ID your Capillary Google Workspace account and click Next.\nEnter your password and click Next.\n\nYou will see Intouch Workbench. If you have already signed in to your Capillary Google Workspace account, you will directly see the Workbech page.\n\nLogin with SSO\n\nSign in with a single set of credentials used for multiple applications provided by third party like Microsoft ADFS, G Suite SAML. It allows brand users to login to Capillary Intouch using their Identity Provider (IdP).\n\nSetup SSO with Okta\nWe have partnered with Okta for the integration. SSO setup requires configuring a connection for Capillary in Org’s IdP and setting up Org’s Identity in Okta Adm

In [50]:
from ragas.metrics import answer_relevancy
from ragas import evaluate
from langchain_groq import ChatGroq

In [25]:
llm = ChatGroq(model='qwen-qwq-32b',temperature=0.6,api_key=groq_api_key)

In [51]:
metrics = [answer_relevancy]

results = evaluate(dataset=data,
                   metrics=metrics,
                   embeddings=embeddings,
                   llm=llm)

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

In [52]:
print(results)

{'answer_relevancy': 0.6659}
